### Create subset of all gravity mains that connect directly to a given outfall

In [1]:
# imports
from trace_sewershed import *
import pandas as pd

network_path = "data/Sewer Gravity Main - 4326.gpkg"

multiple = True
upstream_only = False
downstream_only = False
verbose = True

outfall_file = 'Outfalls_ID.csv'
id_field = 'ASSET_ID'

outfalls = pd.read_csv(outfall_file)[id_field].tolist()

target_endpoints = outfalls

# target_endpoints = ["JP5428128690"]
outputname_extra = ""
output_dir = f"./"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

Load layers from files (takes a while)

In [2]:
# lines
grav_main = gpd.read_file("data/Tosewer/Sewer Gravity Main - 4326.gpkg")

Prepare hash tables for traversal

In [3]:
# currently only for gravity mains, but could be extended to the entire network

# seperate into separate upstream and downstream tables for each segment
df_grav_ups = grav_main[['Sewer Gravity Asset Identification', 'Sewer Gravity Upstream Maintenance Hole']].rename(columns={'Sewer Gravity Upstream Maintenance Hole': 'node_id', 'Sewer Gravity Asset Identification': 'segment_id'})
df_grav_dwns = grav_main[['Sewer Gravity Asset Identification', 'Sewer Gravity Downstream Maintenance Hole']].rename(columns={'Sewer Gravity Downstream Maintenance Hole': 'node_id', 'Sewer Gravity Asset Identification': 'segment_id'})

# combine to a single dataframe for all segment-node connections
df_nodes = pd.concat([df_grav_ups, df_grav_dwns], ignore_index=True)

# group segments by node_id for node-segment lookup and vice-versa for segment-node lookup
node_lookup = df_nodes.groupby('node_id')['segment_id'].apply(set).to_dict()
segment_lookup = df_nodes.groupby('segment_id')['node_id'].apply(set).to_dict()

In [4]:
from trace_sewershed import dfs

visited_list = []
edges_list = []

visited_list, edges_list = dfs(segment_lookup, node_lookup, target_endpoint, visited_list, edges_list)

print("List of Nodes:")
edges_list

List of Nodes:


['SL23453',
 'SL23453',
 'SL23452',
 'SL23451',
 'SL21200',
 'SL21199',
 'SL21199',
 'SL21198',
 'SL21198',
 'SL21166',
 'SL21166',
 'SL21200',
 'SL23451',
 'SL23452']

In [43]:
out_df = pd.DataFrame(data={'segmentID': edges_list}, )

out_df.to_csv(f'catchment_{target_endpoint}.csv')

### For multiple outfalls

In [12]:
outfalls = pd.read_csv('BC_outfalls.csv')['Asset Identification'].tolist()

target_endpoints = outfalls

In [13]:
all_connected = []

for o in target_endpoints:
        visited_list, edges_list = dfs(segment_lookup, node_lookup, o, [], [])
        
        for e in edges_list:
            all_connected.append({"segment_id": e, "exit_point": o})

all_connected

[{'segment_id': 'SL23382', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23382', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23378', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23378', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23377', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23377', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23375', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23375', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23374', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23374', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23401', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23423', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23423', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23422', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23419', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23420', 'exit_point': 'OF3783406347'},
 {'segment_id': 'SL23420', 'exit_point': 'OF3783406347'},
 {'segment_id'

In [ ]:
out_df = pd.DataFrame.from_dict(all_connected)

out_df.to_csv(f'catchment_{"BC_outfalls"}.csv')